In [ ]:
import sounddevice as sd
import librosa
import numpy as np
import joblib
import noisereduce as nr  # Gürültü azaltma için

# Modeli ve scaler'ı yükle
model = joblib.load("../Deneme/VoiceModel.pkl")
scaler = joblib.load("../Deneme/Scaler.pkl")  # Eğitilen scaler

# Tahmin için kullanılacak etiketler
labels = ['elif','irem','nazlı']  # Eğitim sırasında kullanılan kişi isimleri

# Etiket eşleştirme fonksiyonu
def map_label_to_person(predicted_label):
    return labels[predicted_label[0]]

# Özellik çıkarma fonksiyonu
def extract_features_from_audio(y, sr):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)

    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y))
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=y))
    rms = np.mean(librosa.feature.rms(y=y))

    # Özellikleri birleştir
    features = np.concatenate([mfcc_mean, mfcc_std, chroma_mean, 
                                [spectral_centroid, spectral_bandwidth, spectral_rolloff, zcr, rms]])
    return features

# Ses toplama fonksiyonu
def collect_audio():
    print("Konuşmaya başlayabilirsiniz. Susunca tahmin yapılacak...")
    duration = 0  # Ses uzunluğu
    silence_threshold = 0.02  # Sessizlik eşiği (0 ile 1 arasında)
    buffer = []
    sr = 16000  # Sabit örnekleme oranı

    def callback(indata, frames, time, status):
        nonlocal buffer, duration
        if status:
            print(f"Durum: {status}")
        audio_chunk = indata[:, 0]
        rms = np.sqrt(np.mean(audio_chunk**2))  # RMS ile gürültü seviyesi ölçümü
        if rms > silence_threshold:
            buffer.extend(audio_chunk)
            duration += frames / sr
        elif len(buffer) > 0:  # Sessizlik algılanınca dur
            raise sd.CallbackStop()

    with sd.InputStream(callback=callback, channels=1, samplerate=sr, blocksize=sr):
        try:
            sd.sleep(6000)  # Maksimum 60 saniye dinleme
        except sd.CallbackStop:
            pass

    return np.array(buffer), sr

# Ana tahmin fonksiyonu
def main():
    # Ses toplama
    audio_data, sr = collect_audio()

    # Gürültü azaltma
    reduced_noise = nr.reduce_noise(y=audio_data, sr=sr)

    # Özellik çıkarma
    features = extract_features_from_audio(reduced_noise, sr)

    # Özellikleri normalize et
    features_normalized = scaler.transform([features])

    # Tahmin yap
    predicted_label = model.predict(features_normalized)
    predicted_person = map_label_to_person(predicted_label)
    print(f"Tahmin Edilen Kişi: {predicted_person}")

if __name__ == "__main__":
    main()


Konuşmaya başlayabilirsiniz. Susunca tahmin yapılacak...


c:\Users\iremm\.conda\envs\BTK-AI\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 97 features, but KNeighborsClassifier is expecting 98 features as input.